#### File used to initially develop [MLBench_eval.py](./MLBench_eval.py)


In [2]:
#stdlib
# import sys
import os
# import json
# import warnings
# warnings.filterwarnings('ignore')


# relative import error with python modules and what not, the path needs to change according to cwd :)
# project_root = os.path.abspath(os.path.join(os.getcwd(), "../../.."))
# if project_root not in sys.path:
#     sys.path.append(project_root)
    # Load model directly

#hf
from transformers import AutoTokenizer, AutoModel #AutoModelForCausalLM
# from datasets import Dataset
from huggingface_hub import login

#pytorch
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel

# import numpy as np

#a2a
# import lib.embeddings as emb

In [3]:
login(os.environ.get("HF_TOKEN"))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /home/turbouser44/.cache/huggingface/token
Login successful


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
torch.cuda.empty_cache()

In [9]:
print(torch.cuda.memory_summary())

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |      0 B   |      0 B   |      0 B   |      0 B   |
|       from large pool |      0 B   |      0 B   |      0 B   |      0 B   |
|       from small pool |      0 B   |      0 B   |      0 B   |      0 B   |
|---------------------------------------------------------------------------|
| Active memory         |      0 B   |      0 B   |      0 B   |      0 B   |
|       from large pool |      0 B   |      0 B   |      0 B   |

In [17]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


sentences = ['This is an example sentence yes.', 'Each sentence is converted yes.', 'sometimes it may occur that the turbo encabulator']

tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')


# Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# # Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

# # Perform pooling
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
print(sentence_embeddings.shape)

# # Normalize embeddings
sentence_embeddings = F.normalize(sentence_embeddings, p=2)
print(sentence_embeddings.shape)
# print("Sentence embeddings:")
# print(sentence_embeddings.shape)


/home/turbouser44/miniconda3/envs/a3a/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


torch.Size([3, 384])
torch.Size([3, 384])


In [151]:
model_output[0].shape

torch.Size([3, 13, 384])

In [175]:
model_output[0][0]

tensor([[ 0.0467, -0.0398,  0.2078,  ...,  0.0575, -0.1750, -0.3159],
        [ 0.7193,  0.7874,  0.1290,  ...,  0.3034,  0.7406,  0.4794],
        [ 0.0811,  0.1369,  0.0521,  ...,  0.1514,  0.5813, -0.1111],
        ...,
        [-0.1886,  0.2391,  0.4988,  ...,  0.1194,  0.3015, -0.3285],
        [-0.2495,  0.3911,  0.5236,  ...,  0.0672,  0.5542, -0.3707],
        [-0.2521,  0.3885,  0.4938,  ...,  0.0400,  0.5937, -0.3671]])

In [176]:
before_mask_sum = torch.sum(model_output[0][0],dim=1)

In [189]:
# batch == sentence?
expanded_attn_mask = (
    encoded_input["attention_mask"].unsqueeze(-1).expand(model_output[0].shape)
)
first_sent_emb = model_output[0][0]

In [190]:
expanded_attn_mask.shape, first_sent_emb.shape

(torch.Size([3, 13, 384]), torch.Size([13, 384]))

In [193]:
expanded_attn_mask_first_batch[0]

tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])

In [183]:
# sample meanpool
after_mask_sum = torch.sum(first_sent_emb*expanded_attn_mask_first_batch[0],dim=1)
before_mask_sum,after_mask_sum

(tensor([ 1.3301, -7.0392, -1.6298, -4.5336, -0.1929, -0.3943,  2.2787,  5.9446,
          7.4758,  2.3501,  3.7298,  1.7948,  1.2353]),
 tensor([ 1.3301, -7.0392, -1.6298, -4.5336, -0.1929, -0.3943,  2.2787,  5.9446,
          7.4758,  0.0000,  0.0000,  0.0000,  0.0000]))

In [217]:
before_mask_sum

tensor([ 1.3301, -7.0392, -1.6298, -4.5336, -0.1929, -0.3943,  2.2787,  5.9446,
         7.4758,  2.3501,  3.7298,  1.7948,  1.2353])

In [216]:
torch.clamp(after_mask_sum/expanded_attn_mask[0].sum(1),min=-1e-5)

tensor([ 3.4637e-03, -1.0000e-05, -1.0000e-05, -1.0000e-05, -1.0000e-05,
        -1.0000e-05,  5.9342e-03,  1.5481e-02,  1.9468e-02,         nan,
                nan,         nan,         nan])

In [208]:
after_mask_sum/expanded_attn_mask[0].sum(1)

tensor([ 0.0035, -0.0183, -0.0042, -0.0118, -0.0005, -0.0010,  0.0059,  0.0155,
         0.0195,     nan,     nan,     nan,     nan])

In [199]:
expanded_attn_mask[0].sum(1)

tensor([384, 384, 384, 384, 384, 384, 384, 384, 384,   0,   0,   0,   0])

In [94]:
model_output.last_hidden_state[:,0,:].mean()
# sentence_embeddings

tensor(0.0013)

In [86]:
sentence_embeddings.mean()

tensor(9.3265e-05)

In [38]:
model_output.last_hidden_state.shape

torch.Size([3, 13, 384])

In [45]:
model_output[0].shape, model_output[1].shape

(torch.Size([3, 13, 384]), torch.Size([3, 384]))

In [46]:
# encoded_input['attention_mask'].unsqueeze(-1).expand(model_output[0].size()).float()[1][0].shape
sentence_embeddings.shape

torch.Size([3, 384])

In [65]:
encoded_input['input_ids']

tensor([[  101,  2023,  2003,  2019,  2742,  6251,  2748,  1012,   102,     0,
             0,     0,     0],
        [  101,  2169,  6251,  2003,  4991,  2748,  1012,   102,     0,     0,
             0,     0,     0],
        [  101,  2823,  2009,  2089,  5258,  2008,  1996, 15386,  4372,  3540,
         28507,  4263,   102]])

In [59]:
??encoded_input.word_ids


Signature: encoded_input.word_ids(batch_index: int = 0) -> List[Optional[int]]
Source:   
    def word_ids(self, batch_index: int = 0) -> List[Optional[int]]:
        """
        Return a list mapping the tokens to their actual word in the initial sentence for a fast tokenizer.

        Args:
            batch_index (`int`, *optional*, defaults to 0): The index to access in the batch.

        Returns:
            `List[Optional[int]]`: A list indicating the word corresponding to each token. Special tokens added by the
            tokenizer are mapped to `None` and other tokens are mapped to the index of their corresponding word
            (several tokens will be mapped to the same word index if they are parts of that word).
        """
        if not self._encodings:
            raise ValueError(
                "word_ids() is not available when using non-fast tokenizers (e.g. instance of a `XxxTokenizerFast`"
                " class)."
            )
        return self._encodings[ba

In [112]:
encoded_input.word_ids()

[None, 0, 1, 2, 3, 4, 5, 6, None]

In [62]:
# Flatten the input IDs
word_ids = encoded_input["input_ids"].tolist()
flat_word_ids = [word_id for sentence in word_ids for word_id in sentence if word_id is not None]

# Convert IDs to tokens
tokens = tokenizer.convert_ids_to_tokens(flat_word_ids)
print("Tokens:", tokens)

Tokens: ['[CLS]', 'this', 'is', 'an', 'example', 'sentence', 'yes', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[CLS]', 'each', 'sentence', 'is', 'converted', 'yes', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[CLS]', 'sometimes', 'it', 'may', 'occur', 'that', 'the', 'turbo', 'en', '##ca', '##bula', '##tor', '[SEP]']


In [67]:
model_output.keys(), model_output[0].shape, model_output[1].shape,

(odict_keys(['last_hidden_state', 'pooler_output']),
 torch.Size([3, 13, 384]),
 torch.Size([3, 384]))

In [ ]:
encoded_input.keys(), encoded_input[0], encoded_input[1]